An attempt to look for all subreddits related to McDonalds.

## Finding related Subreddits with keyword

In [1]:
import praw
import networkx as nx
import matplotlib.pyplot as plt
import csv
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime
import time
import pandas as pd

reddit = praw.Reddit(
    client_id = os.environ.get("REDDIT_ID"), # See slide 9 & 10 of Lab 2 Guide
    client_secret = os.environ.get("REDDIT_SECRET"), # See slide 9 & 10 of Lab 2 Guide
   username = os.environ.get("REDDIT_USER"), # See slide 4 (step 3) of Lab 2 Guide
   password = os.environ.get("REDDIT_PASS"),  # See slide 4 (step 3) of Lab 2 Guide
   user_agent = os.environ.get("PROJ_NAME") # See slide 9 & 10 of Lab 2 Guide
)

## Testing

In [10]:
# Set the search term
search_term = "Frontier Airlines Inc."

# Create a set to hold unique subreddit names
subreddit_names = set()

# Search for subreddits mentioning "McDonald" in their titles
for subreddit in reddit.subreddits.search(search_term, limit=100):
    subreddit_names.add(subreddit.display_name)

# Print the unique subreddit names
for subreddit in subreddit_names:
    print(subreddit)

frontierairlines
frontiercadetprogram
HawaiianAirlines
BFCH_info
FlyPlayAirlines
gowildpass
Frontiercadets
shade3d
ElitePS


In [4]:
# Define the search term to find subreddits
search_term = "McDonald"

# Search for subreddits with the term "McDonald"
subreddits = reddit.subreddits.search(search_term, limit=1000)  # Adjust limit as needed

# Filter subreddits that seem US-centric or mention locations in their description
mcdonalds_us_subreddits = []

for subreddit in subreddits:
    description = subreddit.public_description.lower()
    # Check if the subreddit name or description seems related to the US
    if "usa" in description or "us" in description or "america" in description:
        mcdonalds_us_subreddits.append(subreddit.display_name)

# Print the list of relevant subreddits
print("Subreddits related to McDonald's in the US:")
for sub in mcdonalds_us_subreddits:
    print(sub)


Subreddits related to McDonald's in the US:
juicyscoopsnark
McLounge
nostalgia
fastfood
todayilearned
teenagers
politics
news
antiwork
doordash_drivers
worldnews
britishproblems
ireland
vegan
nba
PokemonTCG
conspiracy
nfl
CasualUK
NoStupidQuestions
dogecoin
business
McDonaldsToys
CampCope
99rideau
Scrubs
Redhair
Corporatefail
KingCrimson
Fridge
teh_ronald
SnowFall
ranranru
GeroothMcNugget
PopulationMaps
soggy_pickles
genshin_mcdonalds
SanBernardinoCounty
mcdonaldsistersnark
hereisthesoda
McManagers
McDonaldsLore
McDonaldsAusStories
mcdonaldsmenu
McdonaldsWorker
CopyCatRecipes


### Notes:
- Possible subs:
  - McLounge
  - fastfood
  - McDonaldsToys
  - McDonalds
  - McDonaldsEmployees

## Scrape Data from Reddit

In [2]:
# Define parameters
subreddit_name = "McDonalds"
start_date = datetime(2024, 1, 1)
batch_size = 100  # Save data after every 100 posts
batch_posts = []
all_posts = []
last_post = None
output_file = 'mcdonalds_posts_2024_onwards.csv'
first_save = True  # Indicator for writing the header in the first save

# Fetch posts iteratively
while True:
    try:
        # Fetch a batch of posts (up to 100)
        if last_post:
            submissions = reddit.subreddit(subreddit_name).hot(limit=100, params={"after": last_post})
        else:
            submissions = reddit.subreddit(subreddit_name).hot(limit=100)
        
        batch = 0
        
        # Iterate through each submission
        for submission in submissions:
            # Convert the post creation time to a datetime object
            post_date = datetime.utcfromtimestamp(submission.created_utc)
            
            # If the post date is before 2024, stop fetching
            if post_date < start_date:
                batch = 0
                break
            
            # Store the post details
            post_data = {
                'title': submission.title,
                'selftext': submission.selftext,
                'score': submission.score,
                'url': submission.url,
                'created_utc': submission.created_utc,
                'date': post_date,
                'id': submission.id,
                'num_comments': submission.num_comments
            }
            batch_posts.append(post_data)
            batch += 1
            last_post = submission.id  # Update last_post with the latest submission ID
            
            # If batch size is reached, save to CSV
            if len(batch_posts) >= batch_size:
                df = pd.DataFrame(batch_posts)
                
                # Save batch to CSV (append mode after the first save)
                if first_save:
                    df.to_csv(output_file, index=False, mode='w')  # Write with header initially
                    first_save = False
                else:
                    df.to_csv(output_file, index=False, mode='a', header=False)  # Append without header
                
                print(f"Saved {len(batch_posts)} posts to {output_file}")
                batch_posts = []  # Clear the batch list

        # Break the loop if no more posts were added in this batch
        if batch == 0:
            print("No more posts from 2024 onwards.")
            break
        
        # Optional: Sleep to avoid hitting API rate limits
        time.sleep(1)  # Adjust sleep time based on rate limit needs

    except Exception as e:
        print(f"An error occurred: {e}. Retrying...")
        time.sleep(10)  # Wait a bit before retrying in case of error

# Save any remaining posts in the last batch
if batch_posts:
    df = pd.DataFrame(batch_posts)
    if first_save:
        df.to_csv(output_file, index=False, mode='w')  # Write with header if it's the first batch
    else:
        df.to_csv(output_file, index=False, mode='a', header=False)  # Append without header
    
    print(f"Saved the remaining {len(batch_posts)} posts to {output_file}")


Total posts saved: 10000 to mcdonalds_posts_2024_onwards.csv
An error occurred: [Errno 13] Permission denied: 'mcdonalds_posts_2024_onwards.csv'. Retrying...
An error occurred: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /r/McDonalds/hot?after=1cr575e&limit=100&raw_json=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FA7FC97B50>: Failed to resolve 'oauth.reddit.com' ([Errno 11001] getaddrinfo failed)")). Retrying...
An error occurred: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /r/McDonalds/hot?after=1cr575e&limit=100&raw_json=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FA7F7B4F40>: Failed to resolve 'oauth.reddit.com' ([Errno 11001] getaddrinfo failed)")). Retrying...
An error occurred: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (rea

KeyboardInterrupt: 

## Scrape Data with Credentials refresh

In [ ]:
# Define parameters
subreddit_name = "McDonalds"
start_date = datetime(2023, 8, 1)
# end_date = datetime(2024, 5, 16)
batch_size = 100  # Save data after every 100 posts
batch_posts = []
all_posts = []
last_post = None
output_file = 'mcdonalds_posts_aug2023_to_present.csv'
first_save = True  # Indicator for writing the header in the first save
post_count = 0
# Fetch posts iteratively
while True:
    try:
        # Fetch a batch of posts (up to 100)
        if last_post:
            submissions = reddit.subreddit(subreddit_name).hot(limit=100, params={"after": last_post})
        else:
            submissions = reddit.subreddit(subreddit_name).hot(limit=100)

        # keep track of total records saved so far
        post_count += 100
        if post_count % 10000 == 0:
            print(f"Saved {post_count} so far.")
                  
        batch = 0
        
        # Iterate through each submission
        for submission in submissions:
            # Convert the post creation time to a datetime object
            post_date = datetime.utcfromtimestamp(submission.created_utc)

            # If post is after may 2024, skip
            # if post_date > end_date:
            #     last_post = submission.id
            #     continue
            
            # If the post date is before aug 2024, stop fetching
            if post_date < start_date:
                batch = 0
                break
            
            # Store the post details
            post_data = {
                'title': submission.title,
                'selftext': submission.selftext,
                'score': submission.score,
                'url': submission.url,
                'created_utc': submission.created_utc,
                'date': post_date,
                'id': submission.id,
                'num_comments': submission.num_comments
            }
            batch_posts.append(post_data)
            batch += 1
            last_post = submission.id  # Update last_post with the latest submission ID
            
            # If batch size is reached, save to CSV
            if len(batch_posts) >= batch_size:
                df = pd.DataFrame(batch_posts)
                
                # Save batch to CSV (append mode after the first save)
                if first_save:
                    df.to_csv(output_file, index=False, mode='w')  # Write with header initially
                    first_save = False
                else:
                    df.to_csv(output_file, index=False, mode='a', header=False)  # Append without header
                
                print(f"Saved {len(batch_posts)} posts to {output_file}")
                batch_posts = []  # Clear the batch list
        
        # Break the loop if no more posts were added in this batch
        if batch == 0:
            print("No more posts from aug 2023 onwards.")
            break
        
        # Optional: Sleep to avoid hitting API rate limits
        time.sleep(1)  # Adjust sleep time based on rate limit needs

    except Exception as e:
        print(f"An error occurred: {e}. Retrying...")
        time.sleep(10)  # Wait a bit before retrying in case of error
        reddit = praw.Reddit(
            client_id = os.environ.get("REDDIT_ID"), # See slide 9 & 10 of Lab 2 Guide
            client_secret = os.environ.get("REDDIT_SECRET"), # See slide 9 & 10 of Lab 2 Guide
           username = os.environ.get("REDDIT_USER"), # See slide 4 (step 3) of Lab 2 Guide
           password = os.environ.get("REDDIT_PASS"),  # See slide 4 (step 3) of Lab 2 Guide
           user_agent = os.environ.get("PROJ_NAME") # See slide 9 & 10 of Lab 2 Guide
        )

# Save any remaining posts in the last batch
if batch_posts:
    df = pd.DataFrame(batch_posts)
    if first_save:
        df.to_csv(output_file, index=False, mode='w')  # Write with header if it's the first batch
    else:
        df.to_csv(output_file, index=False, mode='a', header=False)  # Append without header
    
    print(f"Saved the remaining {len(batch_posts)} posts to {output_file}")


Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 posts to mcdonalds_posts_aug2023_to_present.csv
Saved 100 post

### Resuming from Breakpoint

In [10]:
df.sort_values(by='date').head(5)

,title,selftext,score,url,created_utc,date,id,num_comments
49,McDonald's launches Grimace Shake in Canada,,22,https://www.verdictfoodservice.com/news/mcdona...,1.715704e+09,2024-05-14 16:20:30,1crw6kf,16
37,McDonald's launches Grimace Shake in Canada,,25,https://www.verdictfoodservice.com/news/mcdona...,1.715704e+09,2024-05-14 16:20:30,1crw6kf,16
39,McDonald's launches Grimace Shake in Canada,,23,https://www.verdictfoodservice.com/news/mcdona...,1.715704e+09,2024-05-14 16:20:30,1crw6kf,16
41,McDonald's launches Grimace Shake in Canada,,23,https://www.verdictfoodservice.com/news/mcdona...,1.715704e+09,2024-05-14 16:20:30,1crw6kf,16
43,McDonald's launches Grimace Shake in Canada,,23,https://www.verdictfoodservice.com/news/mcdona...,1.715704e+09,2024-05-14 16:20:30,1crw6kf,16
...,...,...,...,...,...,...,...,...
64,Richard and Maurice McDonald open the 1st McDo...,,6,https://samoaglobalnews.com/richard-and-mauric...,1.715796e+09,2024-05-15 18:01:21,1csramq,0
32,Richard and Maurice McDonald open the 1st McDo...,,2,https://samoaglobalnews.com/richard-and-mauric...,1.715796e+09,2024-05-15 18:01:21,1csramq,0
16,Richard and Maurice McDonald open the 1st McDo...,,4,https://samoaglobalnews.com/richard-and-mauric...,1.715796e+09,2024-05-15 18:01:21,1csramq,0
70,Richard and Maurice McDonald open the 1st McDo...,,6,https://samoaglobalnews.com/richard-and-mauric...,1.715796e+09,2024-05-15 18:01:21,1csramq,0


In [9]:
df.shape

(100, 8)

## EDA on Rach's comments
Taking a look at scraped comments that were analysed using Vadar

In [6]:
import pandas as pd
import glob

# Define the path to the folder containing the CSV files
folder_path = './macdonald_us/*.csv'  # Update this path as needed

# Use glob to get all CSV file paths in the folder
csv_files = glob.glob(folder_path)

# Create a list to hold the individual DataFrames
dataframes = []

# Iterate through the list of CSV files and read each into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame or save it to a new CSV file
print(combined_df.head())  # Display the first few rows

# Optional: Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_sentiment_comments.csv', index=False)


  comment_id   parent_id                                       comment_body  \
0    jlwxnb7  t3_13tt3hh  The US is the home base. They add on to the me...   
1    jlwygrp  t3_13tt3hh  I'm not sure, I've never traveled to another c...   
2    jlwyt4h  t3_13tt3hh  Because Americans go someplace other than McDo...   
3    jlwxxt8  t3_13tt3hh  Hold on OP, I am going to call up the ghost of...   
4    jlwy4jz  t3_13tt3hh  Because McDonalds has a million competitors in...   

                 author  score   created_utc sentiment  sentiment_score  \
0               tsukiii     50  1.685255e+09   Neutral           0.0000   
1            JimBones31     40  1.685256e+09  Negative          -0.2411   
2     VeronicaMarsupial     35  1.685256e+09  Positive           0.4172   
3  Accomplished-Park480     25  1.685255e+09  Negative          -0.3182   
4              HotSteak     11  1.685255e+09  Negative          -0.8176   

  post_title post_id  
0        NaN     NaN  
1        NaN     NaN  
2    

In [12]:
combined_df.iloc[2].comment_body

"Because Americans go someplace other than McDonald's when they want interesting food. McDonald's is for people who don't want pizzazz."